# First Project (e-commerce)

Здравствуйте дорогие кураторы! 

Ниже выполнеенное мной задание в рамках промежуточного проекта по анализу данных.
Это моя первая подобная работа, поэтому выполнял ее читая статьи, гугля наиболее верные варинаты решения и т.д. 
Сделал по максимуму и очень надеюсь, что получился качественный анализ.

Очень буду рад, если сможете подсветить ошибки, неточности, более эффективные способы решения (а такие, уверен, что есть). Это очень поможет мне вырасти над собой и научиться новому, т.к. очень хочу сменить сферу деятельности.

Заранее спасибо!

С уважением,
Никита Коротков
ns.korotkov@yandex.ru

In [1]:
# Импортируем необходимые библиотеки.

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import requests as rq
from urllib.parse import urlencode

import calendar
from datetime import timedelta

%matplotlib inline

In [2]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
customers = 'https://disk.yandex.ru/d/FUi5uSd6BfG_ig'  # Таблица с уникальными идентификаторами пользователей
olist_orders = 'https://disk.yandex.ru/d/t9Li4JOfgxuUrg' # Таблица заказов
olist_items = 'https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw' # Товарные позиции, входящие в заказы

# Получаем загрузочные ссылки
download_url_customers = rq.get(base_url + urlencode(dict(public_key=customers))).json()['href']
download_url_olist_items = rq.get(base_url + urlencode(dict(public_key=olist_items))).json()['href']
download_url_olist_orders = rq.get(base_url + urlencode(dict(public_key=olist_orders))).json()['href']

In [3]:
# Загружаем файлы и сохраняем их.
download_response = rq.get(download_url_customers)
customers_df = pd.read_csv(download_url_customers)

download_response = rq.get(download_url_olist_items)
olist_items_df = pd.read_csv(download_url_olist_items,
                            parse_dates=['shipping_limit_date'])


download_response = rq.get(download_url_olist_orders)
olist_orders_df = pd.read_csv(download_url_olist_orders, 
                              parse_dates=['order_purchase_timestamp', 
                                          'order_approved_at', 
                                          'order_delivered_carrier_date', 
                                          'order_delivered_customer_date', 
                                          'order_estimated_delivery_date'])

### Customers Data Frame / Таблица с уникальными идентификаторами пользователей

#### Описание данных:
- `customer_id` — позаказный идентификатор пользователя (аналог номера паспорта)
- `customer_unique_id` —  уникальный идентификатор пользователя
- `customer_zip_code_prefix` —  почтовый индекс пользователя
- `customer_city` —  город доставки пользователя
- `customer_state` —  штат доставки пользователя

In [4]:
customers_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
customers_df.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [6]:
customers_df.shape

(99441, 5)

### Orders Data Frame / Таблица заказов

#### Описание данных:
- `order_id` —  уникальный идентификатор заказа (номер чека)
- `customer_id` —  позаказный идентификатор пользователя (аналог номера паспорта)
- `order_status` —  статус заказа
- `order_purchase_timestamp` —  время создания заказа
- `order_approved_at` —  время подтверждения оплаты заказа
- `order_delivered_carrier_date` —  время передачи заказа в логистическую службу
- `order_delivered_customer_date` —  время доставки заказа
- `order_estimated_delivery_date` —  обещанная дата доставки

In [7]:
olist_orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [8]:
olist_orders_df.shape

(99441, 8)

In [9]:
olist_orders_df.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

### Order Items Data Frame / Товарные позиции, входящие в заказы

#### Описание данных:
- `order_id` —  уникальный идентификатор заказа (номер чека)
- `order_item_id` —  идентификатор товара внутри одного заказа
- `product_id` —  ид товара (аналог штрихкода)
- `seller_id` — ид производителя товара
- `shipping_limit_date` —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике
- `price` —  цена за единицу товара
- `freight_value` —  вес товара

In [10]:
olist_items_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [11]:
customers_df.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [12]:
customers_df.shape

(99441, 5)

### Создаем дата сеты для решения последующих задач.
Для решения последующих задач нам необходимо объединить датафреймы выше и создать 2 варианта:
1. со всеми заказами, где просто будут смердженые датафреймы.
2. со всеми успешными покупками, где мы отфильтруем данные и оставим только те, которые можно назвать успешными покупками.

In [13]:
# Мерджим первые 2 датафрейма left join-ом

df = olist_orders_df.merge(customers_df, on='customer_id', how='left')

In [14]:
# Убедимся, что нет заказов без unique_id пользователя после мерджа.

df.customer_unique_id.isna().value_counts()

False    99441
Name: customer_unique_id, dtype: int64

In [15]:
# Делаем финальный merged data frame со всеми сделанными заказами для последующего выполнения заданий.

orders_df = df.merge(olist_items_df, on='order_id', how='left')

Узнаем, какие есть статусы заказа. Видим, что некоторые заказы были отмененены или недоступны для заказа, 
поэтому их стоит убрать из списка успешных покупок.

- `created` —  создан
- `approved` —  подтверждён
- `invoiced` —  выставлен счёт
- `processing` —  в процессе сборки заказа
- `shipped` —  отгружен со склада
- `delivered` —  доставлен пользователю
- `unavailable` —  недоступен
- `canceled` —  отменён

Убираем из списка успешных покупок **отмененные** и **недоступные заказы**, а так же те, по которым **нет факта оплаты**. Остаются только те покупки, которые мы с определенной долей уверености можем назвать успешными.

In [16]:
orders_df.order_status.value_counts()

delivered      110197
shipped          1186
canceled          706
unavailable       610
invoiced          361
processing        357
created             5
approved            3
Name: order_status, dtype: int64

In [17]:
purchases_df = orders_df \
    .query('order_status != "unavailable" & order_status != "canceled" & order_approved_at != "NaT"')

Видим, что есть 3 заказа, которые не были доставлены до покупателя, нет информации по цене и кол-ву проданных товаров. Учитывая все это, и то, что это 3 заказа из более чем 120к заказов, считаю правильным их также убрать из финального датасета для чистоты данных.

In [18]:
purchases_df.loc[purchases_df.seller_id.isna()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
26478,a68ce1686d536ca72bd2dadc4b8671e5,d7bed5fac093a4136216072abaf599d5,shipped,2016-10-05 01:47:40,2016-10-07 03:11:22,2016-11-07 16:37:37,NaT,2016-12-01,f15a952dfc52308d0361288fbf42c7b3,91250,porto alegre,RS,NaN,NaN,NaN,NaT,NaN,NaN
65606,2ce9683175cdab7d1c95bcbb3e36f478,b2d7ae0415dbbca535b5f7b38056dd1f,invoiced,2016-10-05 21:03:33,2016-10-06 07:46:39,NaT,NaT,2016-11-25,6a2da481aa7827b951175772a0fe8bb8,16012,aracatuba,SP,NaN,NaN,NaN,NaT,NaN,NaN
79688,e04f1da1f48bf2bbffcf57b9824f76e1,0d00d77134cae4c58695086ad8d85100,invoiced,2016-10-05 13:22:20,2016-10-06 15:51:38,NaT,NaT,2016-11-29,8886115442775dd8a20c2dcc921c7cc8,88070,florianopolis,SC,NaN,NaN,NaN,NaT,NaN,NaN


In [19]:
# Делаем финальный data frame только с успешными покупками, убрав данные о трех покупках, о которых писл выше.

final_purchases_df = purchases_df.loc[purchases_df.seller_id.notna()]
final_purchases_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


# Задание 1.

Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов)

Как выяснилось, у каждгого unique_id было несколько customer_id (т.е. несколько заказов), поэтому в дальнейшим используем эту колонку для группировки по уникальным юзерам. Для группировки по уникальным заказам (а не по покупкам внутри заказа) импользуем колонку order_id. Фильтруем тех, у кого при группировке оказалось по 1 заказу.

In [21]:
final_purchases_df \
    .groupby(['customer_unique_id'], as_index=False) \
    .agg({'order_id': 'count'}) \
    .rename(columns={'order_id': 'orders_number'}) \
    .query('orders_number == 1') \
    .shape[0]

83168

### Ответ: 
Один раз покупку совершило 83168 человек.

# Задание 2.

Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (5 баллов)

В данных мы снова видим раcхождения, заказы со статусом delivered, но без даты доставки до пользователя.
Есть предположение, что это может быть самовывоз, поэтому их мы также будем учитывать как доставленные.

In [22]:
orders_df[orders_df.order_status == 'delivered'].loc[orders_df.order_delivered_customer_date.isna()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
3376,2d1e2d5bf4dc7227b3bfebb81328c15f,ec05a6d8558c6455f0cbbd8a420ad34f,delivered,2017-11-28 17:44:07,2017-11-28 17:56:40,2017-11-30 18:12:23,NaT,2017-12-18,13467e882eb3a701826435ee4424f2bd,18520,cerquilho,SP,1.0,a50acd33ba7a8da8e9db65094fa990a4,8581055ce74af1daba164fdbd55a40de,2017-12-04 17:56:40,117.30,17.53
23485,f5dd62b788049ad9fc0526e3ad11a097,5e89028e024b381dc84a13a3570decb4,delivered,2018-06-20 06:58:43,2018-06-20 07:19:05,2018-06-25 08:05:00,NaT,2018-07-16,2f17c5b324ad603491521b279a9ff4de,18255,quadra,SP,1.0,2167c8f6252667c0eb9edd51520706a1,0bb738e4d789e63e2267697c42d35a2d,2018-06-26 07:19:05,329.00,25.24
49966,2ebdfc4f15f23b91474edf87475f108e,29f0540231702fda0cfdee0a310f11aa,delivered,2018-07-01 17:05:11,2018-07-01 17:15:12,2018-07-03 13:57:00,NaT,2018-07-30,1bd06a0c0df8b23dacfd3725d2dc0bb9,12445,pindamonhangaba,SP,1.0,e7d5464b94c9a5963f7c686fc80145ad,58f1a6197ed863543e0136bdedb3fce2,2018-07-05 17:15:12,139.00,19.07
90294,e69f75a717d64fc5ecdfae42b2e8e086,cfda40ca8dd0a5d486a9635b611b398a,delivered,2018-07-01 22:05:55,2018-07-01 22:15:14,2018-07-03 13:57:00,NaT,2018-07-30,3bc508d482a402715be4d5cf4020cc81,13170,sumare,SP,1.0,e7d5464b94c9a5963f7c686fc80145ad,58f1a6197ed863543e0136bdedb3fce2,2018-07-05 22:15:14,139.00,19.07
94388,0d3268bad9b086af767785e3f0fc0133,4f1d63d35fb7c8999853b2699f5c7649,delivered,2018-07-01 21:14:02,2018-07-01 21:29:54,2018-07-03 09:28:00,NaT,2018-07-24,ebf7e0d43a78c81991a4c59c145c75db,13560,sao carlos,SP,1.0,ec165cd31c50585786ffda6feff5d0a6,8bdd8e3fd58bafa48af76b2c5fd71974,2018-07-05 21:29:54,188.99,15.63
105606,2d858f451373b04fb5c984a1cc2defaf,e08caf668d499a6d643dafd7c5cc498a,delivered,2017-05-25 23:22:43,2017-05-25 23:30:16,NaT,NaT,2017-06-23,d77cf4be2654aa70ef150f8bfec076a6,91330,porto alegre,RS,1.0,30b5b5635a79548a48d04162d971848f,f9bbdd976532d50b7816d285a22bd01e,2017-06-04 23:30:16,179.00,15.00
111339,ab7c89dc1bf4a1ead9d6ec1ec8968a84,dd1b84a7286eb4524d52af4256c0ba24,delivered,2018-06-08 12:09:39,2018-06-08 12:36:39,2018-06-12 14:10:00,NaT,2018-06-26,cce5e8188bf42ffb3bb5b18ff58f5965,7261,guarulhos,SP,1.0,a2a7efc985315e86d4f0f705701b342b,ed4acab38528488b65a9a9c603ff024a,2018-06-18 12:30:35,110.99,9.13
111783,20edc82cf5400ce95e1afacc25798b31,28c37425f1127d887d7337f284080a0f,delivered,2018-06-27 16:09:12,2018-06-27 16:29:30,2018-07-03 19:26:00,NaT,2018-07-19,175378436e2978be55b8f4316bce4811,9431,ribeirao pires,SP,1.0,55bfa0307d7a46bed72c492259921231,343e716476e3748b069f980efbaa294e,2018-07-03 16:29:30,45.90,9.07


Другая сторона медали - отмененные заказы, но с датой доставки. Возможно их просто не принял покупатель или
отменил заказ в самый последний момент. Эи заказы считаем не доставлнными.

In [23]:
orders_df[orders_df.order_status != 'delivered'].loc[orders_df.order_delivered_customer_date.notna()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
3284,1950d777989f6a877539f53795b4c3c3,1bccb206de9f0f25adc6871a1bcf77b2,canceled,2018-02-19 19:48:52,2018-02-19 20:56:05,2018-02-20 19:57:13,2018-03-21 22:03:51,2018-03-09,6a04c8339c44b23e189446216a494569,32071,contagem,MG,1.0,e435ceb7ced9b8446defd858630454ed,2a61c271b1ac763450d846849783e922,2018-02-23 20:56:05,29.99,14.10
9963,dabf2b0e35b423f94618bf965fcb7514,5cdec0bb8cbdf53ffc8fdc212cd247c6,canceled,2016-10-09 00:56:52,2016-10-09 13:36:58,2016-10-13 13:36:59,2016-10-16 14:36:59,2016-11-30,2f96790fba243100730c8d1185dd2f25,5846,sao paulo,SP,1.0,473795a355d29305c3ea6b156833adf5,620c87c171fb2a6dd6e8bb4dec959fc6,2016-10-13 13:36:59,59.90,0.48
9964,dabf2b0e35b423f94618bf965fcb7514,5cdec0bb8cbdf53ffc8fdc212cd247c6,canceled,2016-10-09 00:56:52,2016-10-09 13:36:58,2016-10-13 13:36:59,2016-10-16 14:36:59,2016-11-30,2f96790fba243100730c8d1185dd2f25,5846,sao paulo,SP,2.0,7cd29da0653abeb444703cc5a957f479,620c87c171fb2a6dd6e8bb4dec959fc6,2016-10-13 13:36:59,49.90,29.94
66373,770d331c84e5b214bd9dc70a10b829d0,6c57e6119369185e575b36712766b0ef,canceled,2016-10-07 14:52:30,2016-10-07 15:07:10,2016-10-11 15:07:11,2016-10-14 15:07:11,2016-11-29,59a7bb642795ddc85534fbd5f7360334,20091,rio de janeiro,RJ,1.0,eba7488e1c67729f045ab43fac426f2e,620c87c171fb2a6dd6e8bb4dec959fc6,2016-10-11 15:07:11,249.90,14.03
67569,8beb59392e21af5eb9547ae1a9938d06,bf609b5741f71697f65ce3852c5d2623,canceled,2016-10-08 20:17:50,2016-10-09 14:34:30,2016-10-14 22:45:26,2016-10-19 18:47:43,2016-11-30,f9af3bfab584769f34c314b222d34263,20091,rio de janeiro,RJ,1.0,c66def7098f4d87751f40c3a4855ced1,46dc3b2cc0980fb8ec44634e21d2718e,2016-10-13 14:34:31,99.99,11.23
105596,65d1e226dfaeb8cdc42f665422522d14,70fc57eeae292675927697fe03ad3ff5,canceled,2016-10-03 21:01:41,2016-10-04 10:18:57,2016-10-25 12:14:28,2016-11-08 10:58:34,2016-11-25,b8b8726af116a5cfb35b0315ecef9172,22770,rio de janeiro,RJ,1.0,9c7bdf67b06b419aefb93cfdfc96c55d,ed49da7cadefc454d6a67de249eab240,2016-10-21 16:24:43,21.50,14.11
107662,2c45c33d2f9cb8ff8b1c86cc28c11c30,de4caa97afa80c8eeac2ff4c8da5b72e,canceled,2016-10-09 15:39:56,2016-10-10 10:40:49,2016-10-14 10:40:50,2016-11-09 14:53:50,2016-12-08,d93a22bb52bb2f41a7267b8a02370781,88058,florianopolis,SC,1.0,e24f73b7631ee3fbb2ab700a9acaa258,0cbcee27c791afa0cdcb08587a2013a8,2016-10-14 10:40:50,135.00,18.51


In [24]:
# Делаем дата сет для дальнейшего расчета и визуализации

merged_orders_customer = orders_df[orders_df.order_status != 'delivered']
merged_orders_customer.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09,36edbb3fb164b1f16485364b6fb04c73,98900,santa rosa,RS,1.0,a1804276d9941ac0733cfd409f5206eb,dc8798cbf453b7e0f98745e396cc5616,2017-04-19 13:25:17,49.90,16.05
46,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaT,2018-06-28,08fb46d35bb3ab4037202c23592d1259,13215,jundiai,SP,1.0,c50ca07e9e4db9ea5011f06802c0aea0,e9779976487b77c6d4ac45f75ec7afe9,2018-06-13 04:30:33,14.49,7.87
118,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaT,NaT,2018-08-21,c7f8d7b1fffc946d7069574f74c39f4e,88140,santo amaro da imperatriz,SC,1.0,1522589c64efd46731d3522568e5bc83,28405831a29823802aa22c084cfd0649,2018-08-13 06:15:14,35.00,15.35
148,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaT,NaT,2017-10-03,9f269af9c49244f6ba4a46985a3cfc2e,3436,sao paulo,SP,1.0,61d52f4882421048afd530db53d6f230,fa74b2f3287d296e9fbd2cc80f2d1cf1,2017-09-20 14:30:09,125.90,12.38
185,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaT,2018-02-07,528b011eb7fab3d59c336cc7248eed3a,38600,paracatu,MG,1.0,ee0c1cf2fbeae95205b4aa506f1469f0,cc419e0650a3c5ba77189a1882b7556a,2018-01-18 02:32:30,53.99,15.13


In [25]:
# Добавляем колонки с месяцем и годом обещаной доставки.

merged_orders_customer['order_estimated_delivery_month'] = \
merged_orders_customer.order_estimated_delivery_date.dt.month

merged_orders_customer['order_estimated_delivery_year'] = \
merged_orders_customer.order_estimated_delivery_date.dt.year

merged_orders_customer.head()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order_estimated_delivery_month,order_estimated_delivery_year
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09,36edbb3fb164b1f16485364b6fb04c73,98900,santa rosa,RS,1.0,a1804276d9941ac0733cfd409f5206eb,dc8798cbf453b7e0f98745e396cc5616,2017-04-19 13:25:17,49.90,16.05,5,2017
46,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaT,2018-06-28,08fb46d35bb3ab4037202c23592d1259,13215,jundiai,SP,1.0,c50ca07e9e4db9ea5011f06802c0aea0,e9779976487b77c6d4ac45f75ec7afe9,2018-06-13 04:30:33,14.49,7.87,6,2018
118,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaT,NaT,2018-08-21,c7f8d7b1fffc946d7069574f74c39f4e,88140,santo amaro da imperatriz,SC,1.0,1522589c64efd46731d3522568e5bc83,28405831a29823802aa22c084cfd0649,2018-08-13 06:15:14,35.00,15.35,8,2018
148,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaT,NaT,2017-10-03,9f269af9c49244f6ba4a46985a3cfc2e,3436,sao paulo,SP,1.0,61d52f4882421048afd530db53d6f230,fa74b2f3287d296e9fbd2cc80f2d1cf1,2017-09-20 14:30:09,125.90,12.38,10,2017
185,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaT,2018-02-07,528b011eb7fab3d59c336cc7248eed3a,38600,paracatu,MG,1.0,ee0c1cf2fbeae95205b4aa506f1469f0,cc419e0650a3c5ba77189a1882b7556a,2018-01-18 02:32:30,53.99,15.13,2,2018


Далее группируем по id заказа, году, месяцу и статусу заказа (т.к. данные у нас за несколько лет и месяцев).
Считаем количесвто за каждыей месяц по каждой причине.
Также убираем из дата сета заказы со статусом delivered, т.к. вероятно, что они тоже доставлены, но дата не зафиксирована, о чем я писал ранее.

In [26]:
group_status_mean = merged_orders_customer \
    .groupby(['order_id', 'order_estimated_delivery_month','order_estimated_delivery_year', 'order_status'], 
             as_index=False) \
    .customer_id.count() \
    .groupby(['order_estimated_delivery_year', 'order_status'], as_index=False) \
    .agg({'customer_id': 'mean'}) \
    .rename(columns={'customer_id': 'monthly_mean'}) \
    .sort_values(['order_status','monthly_mean'], ascending=False)

In [30]:
group_status_mean = merged_orders_customer \
    .groupby(['order_estimated_delivery_month','order_estimated_delivery_year', 'order_status'], 
             as_index=False) \
    .order_id.nunique() \
    .groupby(['order_estimated_delivery_year', 'order_status'], as_index=False) \
    .agg({'order_id': 'mean'}) \
    .rename(columns={'order_id': 'monthly_mean'}) \
    .sort_values(['order_status','monthly_mean'], ascending=False)

In [31]:
# Визуализируем полученные данные по среднему количеству недоставленных заказов в месяц по всем возможным статусам. 
# Делаем детализацию за каждый год, чтобы проследить динамику.

fig = px.histogram(group_status_mean, 
             x='order_status', 
             y='monthly_mean', 
             title="Not delivered orders (mean)", 
             labels={'monthly_mean':'Monthly Mean', 'order_status':'Order Status', 'order_estimated_delivery_year':'Year of purchase:'},
             color="order_estimated_delivery_year", 
             barmode="group",
             height=500)
fig.show()

### Ответ: 

Выше визулизированы итоговые данные. Учитывая, что данные взяты за прошедшие периоды, можно предположить, что это не те заказы, которые нахоятся в процессе, а что на этих этапах, что-то пошло не так с процессом обработки и доставки заказа с точки зрения истории.

За 2016 год у нас не силишком много данных, больше всего данных у нас за 2017 год.
Мы можем с уверенностью сказать, что самыми частыми этапами, на которых зависли заказы и не были доставлены:

- 2016 год - shipped;
- 2017 год - approved;
- 2018 год - processing и invoced самый частые.

# Задание 3.

По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)

Берем заранее подготовленный датасет только с успешными покупками.
Создаем колонку с названием дня недели, ориентируясь на колонку order_approved_at.

In [ ]:
final_purchases_df['order_approved_day_name'] = final_purchases_df.order_approved_at.dt.day_name()

In [ ]:
# Группируем данные по id продукта и дню и считаем количество успешных оплат.

best_day = final_purchases_df \
    .groupby(['product_id', 'order_approved_day_name']) \
    .agg({'order_approved_day_name': 'count'}) \
    .rename(columns={'order_approved_day_name': 'count'}) \
    .reset_index()

In [ ]:
best_day.head()

Выводим для каждого product_id лучший день по количеству покупок. Количество значений соответсвует количеству уникальных ID. По некоторым покупкам есть дни с одинаково большим количеством покупок, в этих случая выведено только первое вхождение.

In [ ]:
best_day_final_df = best_day \
    .pivot(index='order_approved_day_name', columns='product_id', values='count') \
    .fillna(0) \
    .idxmax() \
    .to_frame() \
    .rename(columns={0: 'best_day'})

### Ответ:
Ниже мы получили итоговый датафрейм, где по каждому id товара указан день, в который его чаще всего покупают.

In [ ]:
best_day_final_df.head(10)

Вручную проверяем несколько значений на совпадение (ниже отражен один из тестов, все совпали).

In [ ]:
best_day.loc[best_day.product_id == '001b72dfd63e9833e8c02742adf472e3']

In [ ]:
best_day_final_df.loc['001b72dfd63e9833e8c02742adf472e3']

# Задание 4.

Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? 
Не стоит забывать, что внутри месяца может быть не целое количество недель. 
Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

Опять берем подготовленный датафрейм с успешными покупками.

In [ ]:
final_purchases_df.head()

In [ ]:
# Берем нужные колонки из общего датафрейма и добавляем в новую переменную.

customer_orders = final_purchases_df[['order_id', 'customer_unique_id', 'order_approved_at', 'order_status']]
customer_orders.head()

In [ ]:
# Считаем количество уникальных заказов по уникальным юзерам в разные даты.

customer_orders = customer_orders.groupby(['order_id', 'customer_unique_id', 'order_approved_at'], 
                                          as_index=False) \
                                 .order_status.count()

In [ ]:
# Создаем колонки с годом месяцем и количеством недель в месяце для дальнейших расчетов.

customer_orders['order_approved_at_month'] = customer_orders.order_approved_at.dt.month
customer_orders['order_approved_at_year'] = customer_orders.order_approved_at.dt.year
customer_orders['weeks_count'] = customer_orders.order_approved_at.dt.days_in_month / 7

In [ ]:
customer_orders.head()

Группируем данные по пользователю, году и месяцу (+количеству дней в месяце просто чтобы эта колонка осталась) и считаем количество заказов. Ресетим индексы и оставляем, индексами только id пользователей.

In [ ]:
customer_orders = customer_orders.groupby(['customer_unique_id', 'order_approved_at_month', 'order_approved_at_year', 'weeks_count']) \
    .agg({'order_status': 'count'}) \
    .reset_index()

Далее делим сумму заказов каждого пользователя в месяц на количество недель в этом месяце, чтобы узнать средне за неделю.

In [ ]:
customer_orders['orders_per_week'] = customer_orders.order_status / customer_orders.weeks_count

Финальный штрих - группируем данные по id покупателя и месяцу (уже без годов) и считаем среднее средних за неделю.

In [ ]:
week_customer_orders = customer_orders \
    .groupby(['customer_unique_id','order_approved_at_month'], as_index=False) \
    .agg({'orders_per_week':'mean'}) \
    .sort_values('order_approved_at_month') \
    .reset_index(drop=True)

week_customer_orders.head()

Для удобства можем сделать pivot и заменим NaN на '-' (чтобы визуально сделать более красиво и понятно). 
Теперь есть возможность обращаться по user_id, как по индексу и смотреть данные по каждому отдельному юзеру, по всем месяцам.

In [ ]:
week_customer_orders_pivot = week_customer_orders \
    .pivot(index='customer_unique_id', columns='order_approved_at_month', values='orders_per_week') \
    .fillna('-') \
    .rename(columns={
                    1:'January',
                    2:'February',
                    3:'March',
                    4:'April',
                    5:'May',
                    6:'June',
                    7:'July',
                    8:'August',
                    9:'September',
                    10:'October',
                    11:'November',
                    12:'December'
                    })

In [ ]:
week_customer_orders_pivot.head()

Также можем посчитать просто недельное среднее по месяцам, без детализаци по каждому пользователю.
Из задания не совсем очевидно, как именно считать, но так данные не очень информативны.

In [ ]:
week_orders_mean = customer_orders \
    .groupby('order_approved_at_month', as_index=False) \
    .agg({'orders_per_week':'mean'}) \
    .sort_values('order_approved_at_month')

In [ ]:
fig = px.bar(week_orders_mean, 
             x='order_approved_at_month', 
             y='orders_per_week',
             title='Weekly mean purchases per month', 
             labels={'order_approved_at_month':'Month', 'orders_per_week':'Mean of purchases'}, 
             barmode="group")
fig.show()

### Ответ: 
Выше получился итоговый датасет с рассчитанным недельным средним по каждому кастомеру в каждом месяце без деления по годам и округления (т.к. в задании это не требовалось).
Также я визуализировал на графике недельное среднее по месяцам по всем юзерам. Мы видим, что у нас получилось равномерное распределение средних значений.

# Задание 5.

Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. (10 баллов)

#### Описание данных:
- `order_id` —  уникальный идентификатор заказа (номер чека)
- `customer_id` —  позаказный идентификатор пользователя (аналог номера паспорта)
- `order_status` —  статус заказа
- `order_purchase_timestamp` —  время создания заказа
- `order_approved_at` —  время подтверждения оплаты заказа
- `order_delivered_carrier_date` —  время передачи заказа в логистическую службу
- `order_delivered_customer_date` —  время доставки заказа
- `order_estimated_delivery_date` —  обещанная дата доставки

- `customer_id` — позаказный идентификатор пользователя (аналог номера паспорта)
- `customer_unique_id` —  уникальный идентификатор пользователя
- `customer_zip_code_prefix` —  почтовый индекс пользователя
- `customer_city` —  город доставки пользователя
- `customer_state` —  штат доставки пользователя

Используем ранее заготовленный датасет из первого задания, где отфильтрованы только успешные покупки, которые не были отменены.

In [ ]:
final_purchases_df.head()

In [ ]:
# Выбираем из общего датасета только нужные нам колонки.

orders_list = final_purchases_df[['order_id', 'customer_unique_id', 'order_approved_at']]
orders_list.head()

In [ ]:
# Создаем датасет с первым днем покупки по каждому уникальному пользователб для дальнейшего разделения на когорты.

first_purchase_date = final_purchases_df \
    .groupby('customer_unique_id', as_index=False) \
    .agg({'order_approved_at':'min'}) \
    .rename(columns={'order_approved_at':'first_purchase'})

first_purchase_date.head()

In [ ]:
# Мерджим два предыдущих датасета в один объединяя по customer_unique_id.

orders_list = orders_list.merge(first_purchase_date, on='customer_unique_id', how='left')
orders_list.head()

Высчитываем разницу между датой первого заказа и датой каждого заказа, чтобы пронумеровать месяцы по порядку, начиная с первого для каждой когорты.

In [ ]:
orders_list['order_month'] = orders_list.order_approved_at.dt.month
orders_list['first_purchase_month'] = orders_list.first_purchase.dt.month
orders_list['every_purchase_month'] = orders_list.order_month - orders_list.first_purchase_month

In [ ]:
orders_list.first_purchase_month = orders_list.first_purchase_month.apply(lambda x: calendar.month_name[x])

Готовим датасет для визуализации когортного анализа. Фильтруем только январь - декабрь 2017 года, как проcят в задании.

In [ ]:
cogort_df = orders_list.query('first_purchase >= "2017-01-01" & \
                      first_purchase <= "2017-12-31" & \
                      order_approved_at <= "2017-12-31"')

cogort_df = cogort_df.groupby(['first_purchase_month', 'every_purchase_month']) \
    .agg({'order_id':'nunique'}) \
    .reset_index() \
    .pivot(index='first_purchase_month', columns='every_purchase_month', values='order_id')

Считаем retention rate относительно первой покупки, какой процент уникальных клиентов от каждой когорты 
вернулись и купили еще что-то на протяжении 12 месяцев. Понял как сделать без цикла.

In [ ]:
cogort_df = cogort_df.apply(lambda x: x / cogort_df[0])

In [ ]:
# Делаем правильный порядок месяцев в индексах.

cogort_df = cogort_df.reindex(index = ['January',
                           'February',
                           'March',
                           'April',
                           'May',
                           'June',
                           'July',
                           'August',
                           'September',
                           'October',
                           'November',
                           'December'])

Далее добавляем красоты и понятности нашей визуализации. Также переводим значения в проценты.
На графике мы видим, что желтым отмечены самые высокие показатели retention по сравнению с первой покупкой.
Я выбрал такой подход, т.к. мы можем понять какой процент уникальных пользователей вернулся к нам после первой
покупки в каждом последующем месяце.

In [ ]:
cogort_df_color = (cogort_df
            .style
            .set_caption('User Retention by Cohort in 2017 year')  # добавляем подпись
            .background_gradient(cmap='cividis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту

cogort_df_color

### Ответ:

Самый высокий retention rate на 3 месяц после первой покупки = 0.42% и принадлежит он когорте пользователей, которые совершили первую покупку в марте.

# Задание 6.

Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю.

#### Для сегментации есть три подхода:

1. равномерное сегментирование по диапазонам значений (деление на равные бакеты по значениям)
Достоинства метода — сегменты легко выделить. Однако при таком делении распределение покупателей по сегментам будет неравномерным (в один сегмент могут попасть 90% всех клиентов, в другой — 1%).


2. равномерное сегментирование по количеству клиентов (деление на равные бакеты по кол-ву юзеров)
Данное сегментирование позволяет быстро выделить сегменты, так чтобы отсутствовал сильный дисбаланс между группами. Недостаток в данном подходе — плохо выбираются сегменты.


3. сегментирование с фиксированным диапазоном (ручное деление, основывая на знании продукта)
При сегментировании с фиксированным диапазоном необходимо самостоятельно определить границы значений для каждого показателя. Минус - необходимо очень четко понимать продукт, сегмент рынка его цикл, динамику продаж. 

Среди них я бы выбрал первый - разделение по значениям, т.к. с учетом того, что мы обладаем не очень большим объемом данных и ничего не знаем о продукте, он будет самым подходящим. Второй вариант также не подойдет, т.к. будут сложности с делением по количеству заказов. В этом случае у нас очень много клиентов, оторые сделали только 1 заказ.

Также стоит отметить, что в бизнесе довольно часто работает прицип Парето (80% результатов происходят благодаря 20% усилий / или 80% прибыли генерируют 20% топовых клиентов). Соответственно выбираем второй метод, деление по количеству клиентов, как самый устойчивый и делить мы будем на 5 бакетов по 20% (спомощью квантилей), чтобы выделить ту самую приоритетную категорию людей по принципу Парето и все отсальные сегменты.

In [ ]:
def r_score(x):
    if x <= quintiles['recency'][.2]:
        return 5
    elif x <= quintiles['recency'][.4]:
        return 4
    elif x <= quintiles['recency'][.6]:
        return 3
    elif x <= quintiles['recency'][.8]:
        return 2
    else:
        return 1

def fm_score(x, c):
    if x <= quintiles[c][.2]:
        return 1
    elif x <= quintiles[c][.4]:
        return 2
    elif x <= quintiles[c][.6]:
        return 3
    elif x <= quintiles[c][.8]:
        return 4
    else:
        return 5 

In [ ]:
# Мы имеем данные почти за 2 года. При этом данные за полный год у нас только за 2017.

print(f'период покупок от {final_purchases_df.order_approved_at.min()} \
до {final_purchases_df.order_approved_at.max()}')

In [ ]:
final_purchases_df.head()

Группируем датафрейм и считаем цену каждого заказа, каждого юзера на каждую дату. 
Т.к. RFM анализ, как правило, делается на основе данных за полный год (оновываясь на тексте из предложенной статьи), то и для этого задания возьмем данные только за 2017 год, т.к. это единственный год, за который у нас есть полные данные.

In [ ]:
rfm_df = final_purchases_df \
    .groupby(['order_id', 'customer_unique_id', 'order_approved_at'], as_index=False) \
    .agg({'price':'sum'}) \
    .query('order_approved_at >= "2017-01-01" & order_approved_at <= "2017-12-31"')

rfm_df.head()

In [ ]:
print(f'период покупок от {rfm_df.order_approved_at.min()} \
до {rfm_df.order_approved_at.max()}')

In [ ]:
# Для этого упражнения (т.к. данные старые) будем считать, что сегодня + 1 день от самой поздней даты заказа.
today_date = rfm_df.order_approved_at.max() + timedelta(days=1)
today_date

In [ ]:
# Считаем количество дней до "сегодняшнего дня".
rfm_df['days_since_order'] = (today_date - rfm_df.order_approved_at).dt.days
rfm_df.head()

In [ ]:
aggr = {
        'days_since_order': 'min',  # количество дней с последнего заказа (Recency)
        'order_approved_at': 'count', # количество заказов за весь период (Frequency)
        'price': 'sum', # сумма заказов за весь период (Monetary)
       }

rfm_df_final = rfm_df \
    .groupby('customer_unique_id') \
    .agg(aggr) \
    .reset_index() \
    .rename(columns={'days_since_order': 'recency', 'order_approved_at': 'frequency', 'price': 'monetary'})

In [ ]:
rfm_df_final.head()

Объявляем квантили для наших данных, по которым будем делить наши данные.
Обратим внимание, что для нас важно более маленькое значение (80 квантиль) recency (как давно покупали товар), тогда как про frecuency (количество заказов) и monetary (сумма заказов) - наоборот, большее значение приоритетнее (20 квантиль).

In [ ]:
quintiles = rfm_df_final[['recency', 'frequency', 'monetary']].quantile([.2, .4, .6, .8]).to_dict()
quintiles

In [ ]:
# Для каждого юзера создаем колонки с нужным значением, в зависимости от квантиля.

rfm_df_final['R'] = rfm_df_final['recency'].apply(lambda x: r_score(x))
rfm_df_final['F'] = rfm_df_final['frequency'].apply(lambda x: fm_score(x, 'frequency'))
rfm_df_final['M'] = rfm_df_final['monetary'].apply(lambda x: fm_score(x, 'monetary'))

In [ ]:
rfm_df_final.head()

In [ ]:
# Создаем общую колонку RFM сегментирования для каждого пользователя.

rfm_df_final['RFM_score'] = rfm_df_final['R'].map(str) + rfm_df_final['F'].map(str) + rfm_df_final['M'].map(str)
rfm_df_final.head()

In [ ]:
rfm_df_final.R.value_counts()

In [ ]:
rfm_df_final.F.value_counts()

In [ ]:
rfm_df_final.M.value_counts()

Т.к. мы создали 5 сегментов в каждом из 3 критериев, то всего у нас 125 сегментов.
С таким количеством будет довольно сложно работать поэтому их необходимо группировать по другому, чтобы на выходе получилось меньшее количество. Мне нравится разделение, предложенное в статье (прочитал несколько статей, это наиболее оптимальное).

Создаем **словарь с названием разных категорий пользователей**, с учетом их RFM значения.

1. **Champions** - недавно делали покупку и покупают часто;
2. **Loyal Customers** - реулярно делают покупки, следят за акциями;
3. **Potential Loyalist** - недавние клиенты с хорошей частотой покупки;
4. **Recent Customers** - покупали недавно, но покупают не часто;
5. **Promising** - недавно покупали, но много не потратили;
6. **Customers Needing Attention** - "середнячки" по всем показателям;
7. **About To Sleep** - recency и frequency ниже среднего, можем потерять, если не реактивируем;
8. **At Risk** - покупали часто, но давно - необходимо их вернуть;
9. **Can’t Lose Them** - часто покупали, но давно к нам не возвращались;
10. **Hibernating** - заказывали давно и не очень много - можем потерять.

In [ ]:
# Создаем сегменты на основе словаря.

segment_names = {
    r'5[4-5]': 'champions',
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at risk',
    r'[1-2]5': 'can\'t loose',
    r'3[1-2]': 'about to sleep',
    r'33': 'need attention',
    r'[3-4][4-5]': 'loyal customers',
    r'41': 'promising',
    r'51': 'new customers',
    r'[4-5][2-3]': 'potential loyalists',
}

In [ ]:
rfm_df_final['segment'] = rfm_df_final['R'].map(str) + rfm_df_final['F'].map(str)
rfm_df_final['segment'] = rfm_df_final['segment'].replace(segment_names, regex=True)
rfm_df_final.head()

Визуализируем полученные данные.
Графики тоже взял из статьи, но немного изменил цвета. Также сделал так, чтобы подсвечивались и минимальные (красным) значения и максимальные (зеленым) значения.

In [ ]:
# plot the distribution of customers over R and F
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

for i, p in enumerate(['R', 'F']):
    parameters = {'R':'recency', 'F':'frequency'}
    y = rfm_df_final[p].value_counts().sort_index()
    x = y.index
    ax = axes[i]
    bars = ax.bar(x, y, color='silver')
    ax.set_frame_on(False)
    ax.tick_params(left=False, labelleft=False, bottom=False)
    ax.set_title('Distribution of {}'.format(parameters[p]),
                fontsize=14)
    for bar in bars:
        value = bar.get_height()
        if value == y.max():
            bar.set_color('green')
        elif value == y.min():
            bar.set_color('firebrick')
        ax.text(bar.get_x() + bar.get_width() / 2,
                value - 5,
                '{}\n({}%)'.format(int(value), int(value * 100 / y.sum())),
               ha='center',
               va='top',
               color='w')

plt.show()

# plot the distribution of M for RF score
fig, axes = plt.subplots(nrows=5, ncols=5,
                         sharex=False, sharey=True,
                         figsize=(10, 10))

r_range = range(1, 6)
f_range = range(1, 6)
for r in r_range:
    for f in f_range:
        y = rfm_df_final[(rfm_df_final['R'] == r) & (rfm_df_final['F'] == f)]['M'].value_counts().sort_index()
        x = y.index
        ax = axes[r - 1, f - 1]
        bars = ax.bar(x, y, color='silver')
        if r == 5:
            if f == 3:
                ax.set_xlabel('{}\nF'.format(f), va='top')
            else:
                ax.set_xlabel('{}\n'.format(f), va='top')
        if f == 1:
            if r == 3:
                ax.set_ylabel('R\n{}'.format(r))
            else:
                ax.set_ylabel(r)
        ax.set_frame_on(False)
        ax.tick_params(left=False, labelleft=False, bottom=False)
        ax.set_xticks(x)
        ax.set_xticklabels(x, fontsize=8)

        for bar in bars:
            value = bar.get_height()
            if value == y.max():
                bar.set_color('green')
            elif value == y.min():
                bar.set_color('firebrick')
            ax.text(bar.get_x() + bar.get_width() / 2,
                    value,
                    int(value),
                    ha='center',
                    va='bottom',
                    color='k')
fig.suptitle('Distribution of M for each F and R',
             fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# count the number of customers in each segment
segments_counts = rfm_df_final['segment'].value_counts().sort_values(ascending=True)

fig, ax = plt.subplots()

bars = ax.barh(range(len(segments_counts)),
              segments_counts,
              color='silver')
ax.set_frame_on(False)
ax.tick_params(left=False,
               bottom=False,
               labelbottom=False)
ax.set_yticks(range(len(segments_counts)))
ax.set_yticklabels(segments_counts.index)

for i, bar in enumerate(bars):
        value = bar.get_width()
        if segments_counts.index[i] in ['champions', 'loyal customers']:
            bar.set_color('green')
        ax.text(value,
                bar.get_y() + bar.get_height()/2,
                '{:,} ({:}%)'.format(int(value),
                                   round(float(value*100/segments_counts.sum()),2)),
                va='center',
                ha='left'
               )

plt.show()

### Ответ:

На графиках мы видим картину, что у нас очень мало лояльных клиентов и чемпионов и тех, кото мы не можем потерять, т.к. они покупали много, но давно к нам не возвращались. Мы также видим, что у нас очень много пользователей, которые сделали одну покупку и больше ничего не покупали. Это значит, что у нас сильно стредает retention. 

Возможно мы просто очень молодой сервис, а может наш продукт не интересен и нам нужно пересмотреть ассортиментную стратегию, а может что-то третье, вариантов может быть масса. Нужен более подробный анализ для выявления конкретной  проблемы.

Также у нас абсолютно отсутствует прослойка по частоте покупок между 1 и 5 категориями.

Исходя из полученных данных мы можем сделать выводы, что нам необходимо:

1. сделать максимальный акцент на многообещающих (promising), т.к. из них проще всего сделать лояльных или чемпионов;
2. простимулировать новых покупателей совершить повторную покупку;
3. "разбудить" спящих пкупателей, чтобы их не потерять.

Это все поможет нам активировать этих пользователей и нарастить количество лояльных и чемпионов, которых у нас крайне мало.